In [1]:
import tensorflow as tf
import pickle
from keras.layers import (
    Input,
    Dense,
    Input,
    Dense,
    Conv1D,
    Flatten, 
    Embedding,    
    Dropout, 
    Conv1D,
    MaxPooling1D,
    Bidirectional,
    TimeDistributed,
    LSTM)
from keras import Model
%run constants.py

tf.random.set_seed(
    SEED
)
import os
from numpy.random import seed 
seed(SEED)

In [2]:
# Some file name editing for timestep


f = os.path.basename(X_TRAIN_INPUT_SAVE_FILE_VEC_MULTI_TIMESTEP)
X_TRAIN_INPUT_SAVE_FILE_VEC_MULTI_TIMESTEP = os.path.join(DATA_DIR, os.path.splitext(os.path.basename(f))[0] + "_" + str(TIME_STEP) + os.path.splitext(os.path.basename(f))[1]) 
f = os.path.basename(X_TEST_INPUT_SAVE_FILE_VEC_MULTI_TIMESTEP)
X_TEST_INPUT_SAVE_FILE_VEC_MULTI_TIMESTEP = os.path.join(DATA_DIR, os.path.splitext(os.path.basename(f))[0] + "_" + str(TIME_STEP) + os.path.splitext(os.path.basename(f))[1]) 
f = os.path.basename(X_VAL_INPUT_SAVE_FILE_VEC_MULTI_TIMESTEP)
X_VAL_INPUT_SAVE_FILE_VEC_MULTI_TIMESTEP = os.path.join(DATA_DIR, os.path.splitext(os.path.basename(f))[0] + "_" + str(TIME_STEP) + os.path.splitext(os.path.basename(f))[1]) 

f = os.path.basename(Y_TRAIN_INPUT_SAVE_FILE)
Y_TRAIN_INPUT_SAVE_FILE = os.path.join(DATA_DIR, os.path.splitext(os.path.basename(f))[0] + "_" + str(TIME_STEP) + os.path.splitext(os.path.basename(f))[1]) 
f = os.path.basename(Y_TEST_INPUT_SAVE_FILE)
Y_TEST_INPUT_SAVE_FILE = os.path.join(DATA_DIR, os.path.splitext(os.path.basename(f))[0] + "_" + str(TIME_STEP) + os.path.splitext(os.path.basename(f))[1]) 
f = os.path.basename(Y_VAL_INPUT_SAVE_FILE)
Y_VAL_INPUT_SAVE_FILE = os.path.join(DATA_DIR, os.path.splitext(os.path.basename(f))[0] + "_" + str(TIME_STEP) + os.path.splitext(os.path.basename(f))[1]) 

f = os.path.basename(EMBEDDING_MATRIX_SAVE_FILE)
EMBEDDING_MATRIX_SAVE_FILE= os.path.join(DATA_DIR, os.path.splitext(os.path.basename(f))[0] + "_" + str(TIME_STEP) + os.path.splitext(os.path.basename(f))[1]) 
f = os.path.basename(VOCAB_SAVE_FILE)
VOCAB_SAVE_FILE = os.path.join(DATA_DIR, os.path.splitext(os.path.basename(f))[0] + "_" + str(TIME_STEP) + os.path.splitext(os.path.basename(f))[1]) 
f = os.path.basename(TRAIN_CORPORA)
TRAIN_CORPORA = os.path.join(DATA_DIR, os.path.splitext(os.path.basename(f))[0] + "_" + str(TIME_STEP) + os.path.splitext(os.path.basename(f))[1]) 

#Dataset Prep
with open(X_TRAIN_INPUT_SAVE_FILE_VEC_MULTI_TIMESTEP, "rb") as f:
        x_train = pickle.load(f)
with open(Y_TRAIN_INPUT_SAVE_FILE, "rb") as f:
        y_train = pickle.load(f)

with open(X_VAL_INPUT_SAVE_FILE_VEC_MULTI_TIMESTEP, "rb") as f:
        x_val = pickle.load(f)
with open(Y_VAL_INPUT_SAVE_FILE, "rb") as f:
        y_val = pickle.load(f)

with open(X_TEST_INPUT_SAVE_FILE_VEC_MULTI_TIMESTEP, "rb") as f:
        x_test = pickle.load(f)
with open(Y_TEST_INPUT_SAVE_FILE, "rb") as f:
        y_test = pickle.load(f)
with open(EMBEDDING_MATRIX_SAVE_FILE, "rb") as f:
        embedding_matrix = pickle.load(f)
        
embedding_layer = Embedding(input_dim=MAX_VOCAB_SIZE, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH, weights = [embedding_matrix], trainable=False)

In [5]:
def lstm_model_hpam_search(hp):
  

  hp_units = hp.Int('units', min_value=30, max_value=120, step=30)
  hp_dropout = hp.Float('dropout', min_value=0, max_value=0.5, step=0.1)
  hp_optimizer = hp.Choice('optimizer', values=['sgd', 'rmsprop', 'adam'])
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5, 1e-6])

  document_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
  embedding_sequences =embedding_layer(document_input)
  x = LSTM(90)(embedding_sequences)
  x = Dropout(0.3)(x)
  doc_model = Model(document_input, x)
  input_docs = Input(
                shape=(TIME_STEP, MAX_SEQUENCE_LENGTH), name="input_docs", dtype="int32"
            )

  x = TimeDistributed(doc_model, name="token_embedding_model")(input_docs)
  x = LSTM(90)(x)
  x = Dropout(hp_dropout)(x)
  outputs = Dense(1, activation="sigmoid")(x)
  model = Model(input_docs, outputs)
  # model.summary()
  

  # X_indices = Input(shape=(MAX_SEQUENCE_LENGTH,))

  # embeddings = embedding_layer(X_indices)


  # X = LSTM(hp_units)(embeddings)
  
  # X = Dropout(hp_dropout)(X)

  # X = Dense(1, activation='sigmoid')(X)

  # model = Model(inputs=X_indices, outputs=X)
  
  loss = tf.keras.losses.BinaryCrossentropy()
  metrics = tf.metrics.BinaryAccuracy()


  if hp_optimizer == 'sgd':
    optimizer = tf.keras.optimizers.SGD(learning_rate=hp_learning_rate)
  elif hp_optimizer == 'rmsprop':
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=hp_learning_rate)
  elif hp_optimizer == 'adam':
    optimizer = tf.keras.optimizers.Adam(learning_rate=hp_learning_rate)
  else:
      raise
  # opt = tf.keras.optimizers.SGD(learning_rate=hp_learning_rate)

  model.compile(
      optimizer= optimizer,
      loss=loss,
      metrics=metrics,
  )

  return model


In [6]:
import keras_tuner as kt
tuner = kt.Hyperband(lstm_model_hpam_search,
                     objective='val_binary_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='hyperparam_search',
                     project_name='multi_timestep_lstm')

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(x_train, y_train, epochs=50, validation_data=(x_val, y_val), callbacks=[stop_early])
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]



Trial 17 Complete [00h 00m 46s]
val_binary_accuracy: 0.5218977928161621

Best val_binary_accuracy So Far: 0.5218977928161621
Total elapsed time: 00h 00m 46s

Search: Running Trial #18

Value             |Best Value So Far |Hyperparameter
60                |30                |units
0.3               |0.2               |dropout
adam              |rmsprop           |optimizer
0.001             |0.0001            |learning_rate
10                |10                |tuner/epochs
4                 |4                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
2                 |2                 |tuner/round
0013              |0012              |tuner/trial_id

Epoch 5/10
78/78 [==============================] - 10s 98ms/step - loss: 0.6733 - binary_accuracy: 0.5805 - val_loss: 0.6913 - val_binary_accuracy: 0.5219
Epoch 6/10
34/78 [============>.................] - ETA: 3s - loss: 0.6646 - binary_accuracy: 0.5938

KeyboardInterrupt: 

: 

In [ ]:
print(f"""
The hyperparameter search is complete for single LSTM. 
Optimal number of LSTM units are {best_hps.get('units')}
Optimal learning rate for the optimizer is {best_hps.get('learning_rate')}
Optimal optimizer function is {best_hps.get('optimizer')}.
Optimal dropout is {best_hps.get('dropout')}.
""")

In [ ]:
def stacked_lstm_model_hpam_search(hp):


  hp_units = hp.Int('units', min_value=30, max_value=120, step=30)
  hp_dropout = hp.Float('dropout', min_value=0, max_value=0.5, step=0.1)
  hp_optimizer = hp.Choice('optimizer', values=['sgd', 'rmsprop', 'adam'])
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5, 1e-6])
  

  loss = tf.keras.losses.BinaryCrossentropy()
  metrics = tf.metrics.BinaryAccuracy()


  if hp_optimizer == 'sgd':
    optimizer = tf.keras.optimizers.SGD(learning_rate=hp_learning_rate)
  elif hp_optimizer == 'rmsprop':
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=hp_learning_rate)
  elif hp_optimizer == 'adam':
    optimizer = tf.keras.optimizers.Adam(learning_rate=hp_learning_rate)
  else:
      raise

  X_indices = Input(shape=(MAX_SEQUENCE_LENGTH,))

  embeddings = embedding_layer(X_indices)

  X = LSTM(hp_units, return_sequences=True)(embeddings)

  X = Dropout(hp_dropout)(X)

  X = LSTM(hp_units)(X)

  X = Dense(1, activation='sigmoid')(X)

  model = Model(inputs=X_indices, outputs=X)

  model.compile(
      optimizer= optimizer,
      loss=loss,
      metrics=metrics,
  )

  return model

In [ ]:
tuner = kt.Hyperband(stacked_lstm_model_hpam_search,
                     objective='val_binary_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='hyperparam_search',
                     project_name='single_timestep_stacked_lstm')

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(x_train, y_train, epochs=50, validation_data=(x_val, y_val), callbacks=[stop_early])
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]


In [ ]:
print(f"""
The hyperparameter search for stacked is complete. 
Optimal number of LSTM units are {best_hps.get('units')}
Optimal learning rate for the optimizer is {best_hps.get('learning_rate')}
Optimal optimizer function is {best_hps.get('optimizer')}.
Optimal dropout is {best_hps.get('dropout')}).
""")

In [ ]:
def stacked_bilstm_model_hpam_search(hp):


  hp_units = hp.Int('units', min_value=30, max_value=120, step=30)
  hp_dropout = hp.Float('dropout', min_value=0, max_value=0.5, step=0.1)
  hp_optimizer = hp.Choice('optimizer', values=['sgd', 'rmsprop', 'adam'])
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5, 1e-6])
  

  loss = tf.keras.losses.BinaryCrossentropy()
  metrics = tf.metrics.BinaryAccuracy()


  if hp_optimizer == 'sgd':
    optimizer = tf.keras.optimizers.SGD(learning_rate=hp_learning_rate)
  elif hp_optimizer == 'rmsprop':
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=hp_learning_rate)
  elif hp_optimizer == 'adam':
    optimizer = tf.keras.optimizers.Adam(learning_rate=hp_learning_rate)
  else:
      raise

  X_indices = Input(shape=(MAX_SEQUENCE_LENGTH,))

  embeddings = embedding_layer(X_indices)

  X = Bidirectional(LSTM(hp_units, return_sequences=True))(embeddings)

  X = Dropout(hp_dropout)(X)

  X = Bidirectional(LSTM(hp_units))(X)

  X = Dense(1, activation='sigmoid')(X)

  model = Model(inputs=X_indices, outputs=X)

  model.compile(
      optimizer= optimizer,
      loss=loss,
      metrics=metrics,
  )

  return model

In [ ]:
import keras_tuner as kt
tuner = kt.Hyperband(stacked_bilstm_model_hpam_search,
                     objective='val_binary_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='hyperparam_search',
                     project_name='single_timestep_stacked_bilstm')

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(x_train, y_train, epochs=50, validation_data=(x_val, y_val), callbacks=[stop_early])
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]


In [ ]:
print(f"""
The hyperparameter search for bidirectional stacked is complete. 
Optimal number of LSTM units are {best_hps.get('units')}
Optimal learning rate for the optimizer is {best_hps.get('learning_rate')}
Optimal optimizer function is {best_hps.get('optimizer')}.
Optimal dropout is {best_hps.get('dropout')}).
""")

In [ ]:
def bilstm_model_hpam_search(hp):

  hp_units = hp.Int('units', min_value=30, max_value=120, step=30)
  hp_dropout = hp.Float('dropout', min_value=0, max_value=0.5, step=0.1)
  hp_optimizer = hp.Choice('optimizer', values=['sgd', 'rmsprop', 'adam'])
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5, 1e-6])
  

  X_indices = Input(shape=(MAX_SEQUENCE_LENGTH,))

  embeddings = embedding_layer(X_indices)


  X = Bidirectional(LSTM(hp_units))(embeddings)
  
  X = Dropout(hp_dropout)(X)

  X = Dense(1, activation='sigmoid')(X)

  model = Model(inputs=X_indices, outputs=X)
  
  loss = tf.keras.losses.BinaryCrossentropy()
  metrics = tf.metrics.BinaryAccuracy()


  if hp_optimizer == 'sgd':
    optimizer = tf.keras.optimizers.SGD(learning_rate=hp_learning_rate)
  elif hp_optimizer == 'rmsprop':
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=hp_learning_rate)
  elif hp_optimizer == 'adam':
    optimizer = tf.keras.optimizers.Adam(learning_rate=hp_learning_rate)
  else:
      raise
  # opt = tf.keras.optimizers.SGD(learning_rate=hp_learning_rate)

  model.compile(
      optimizer= optimizer,
      loss=loss,
      metrics=metrics,
  )

  return model


In [ ]:
import keras_tuner as kt
tuner = kt.Hyperband(bilstm_model_hpam_search,
                     objective='val_binary_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='hyperparam_search',
                     project_name='single_timestep_bilstm')

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(x_train, y_train, epochs=50, validation_data=(x_val, y_val), callbacks=[stop_early])
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]


In [ ]:
print(f"""
The hyperparameter search for bidirectional LSTM is complete. 
Optimal number of LSTM units are {best_hps.get('units')}
Optimal learning rate for the optimizer is {best_hps.get('learning_rate')}
Optimal optimizer function is {best_hps.get('optimizer')}.
Optimal dropout is {best_hps.get('dropout')}).
""")

In [ ]:
def cnn_model_hpam_search(hp):

  hp_dropout = hp.Float('dropout', min_value=0, max_value=0.5, step=0.1)
  hp_optimizer = hp.Choice('optimizer', values=['sgd', 'rmsprop', 'adam'])
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5, 1e-6])
  hp_filter_num = hp.Choice('filter', values =[100,200,300, 400])
  X_indices = Input(shape=(MAX_SEQUENCE_LENGTH,))

  embeddings = embedding_layer(X_indices)


  X = Conv1D(filters=hp_filter_num, kernel_size=5, padding="valid")(embeddings)

  X = MaxPooling1D()(X)
  
  X = Dropout(hp_dropout)(X)

  X = Flatten()(X)

  X = Dense(1, activation='sigmoid')(X)


  model = Model(inputs=X_indices, outputs=X)
  
  loss = tf.keras.losses.BinaryCrossentropy()
  metrics = tf.metrics.BinaryAccuracy()


  if hp_optimizer == 'sgd':
    optimizer = tf.keras.optimizers.SGD(learning_rate=hp_learning_rate)
  elif hp_optimizer == 'rmsprop':
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=hp_learning_rate)
  elif hp_optimizer == 'adam':
    optimizer = tf.keras.optimizers.Adam(learning_rate=hp_learning_rate)
  else:
      raise
  # opt = tf.keras.optimizers.SGD(learning_rate=hp_learning_rate)

  model.compile(
      optimizer= optimizer,
      loss=loss,
      metrics=metrics,
  )

  return model


In [ ]:
import keras_tuner as kt
tuner = kt.Hyperband(cnn_model_hpam_search,
                     objective='val_binary_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='hyperparam_search',
                     project_name='single_timestep_cnn')

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(x_train, y_train, epochs=50, validation_data=(x_val, y_val), callbacks=[stop_early])
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
print(f"""
The hyperparameter search for CNN is complete. 
# Optimal number of Filters are {best_hps.get('filter')}
Optimal learning rate for the optimizer is {best_hps.get('learning_rate')}
Optimal optimizer function is {best_hps.get('optimizer')}.
Optimal dropout is {best_hps.get('dropout')}).
""")